In [ ]:
<# 

.SYNOPSIS
    A script for SQL Logins audit

.NOTES
                                   __
                               _.-~  )
                    _..--~~~~,'   ,-/     _
                 .-'. . . .'   ,-','    ,' )
               ,'. . . _   ,--~,-'__..-'  ,'
             ,'. . .  (@)' ---~~~~      ,'
            /. . . . '~~             ,-'
           /. . . . .             ,-'
          ; . . . .  - .        ,'
         : . . . .       _     /
        . . . ./           `-.:
       . . . ./  - .          )
    ~---~~~~----~~~~             ~~

.DESCRIPTION
    .Workflow
        [1] Set Variables
        [2] Compare SQL Logins with members of AD Groups, print matches and substract matches from SQL Logins
        [3] Check which of the remaining SQL Logins are for dismissed employees, print them and substract them from SQL Logins
        [4] Print remaining SQL Logins and show which SQL Logins own SQL Jobs and DataBases

    .DataFlow

     [SQL Logins]       [AD Groups members]           [AD Users]              [DBs, Jobs]
        V                        V                        V                        V
       [1] --- SQL_Logins.1 >>> [2] --- SQL_Logins.2 >>> [3] --- SQL_Logins.3 >>> [4]
        \__________________dismissed_____________________/     
    
    .History
        Version 1.0 - initial release
        Version 1.1 - added Dolphin
#>

Import-module -Name ActiveDirectory
Import-module -Name SqlServer

In [ ]:
##[1.1]## Set variables - Server and groups

    #System1
    $SQL_Server = "TestServer"
    $AD_Server = "domain.com"
    $AD_Groups = "Group1", "Group2", "Group3"

In [ ]:
##[1.2]## Set variables - SQL Logins

    $SQL_Logins_All = Get-SqlLogin `
        -logintype "WindowsUser" `
        -ServerInstance "$SQL_Server" | `
        Select -Expand Name
        #Get data like 'domain\user'

In [ ]:
##[1.3]## Set variables - Exclusions

    $Dismissed = @($SQL_Logins_All.Where{$_ -match "DISMISSED\\" })

    $SQL_Logins_Excluded = @(
        "NT ", "\$",
        "Administrator",
        "INVSQLService")

    $SQL_Logins = $SQL_Logins_All.Where{$_ `
        -notin $Dismissed `
        -and $_ -notmatch ($SQL_Logins_Excluded -join "|")}

In [ ]:
##[1.4]## Set variables - Define Functions 

   function Get-GroupMembers() {

    #[1] Pull data: get data like 'CN=Domain Admins,CN=Users,DC=Fabrikam,DC=com'
     $membersAll = Get-ADGroupMember `
        -Identity $AD_group `
        -Recursive `
        -Server $AD_Server | `
        Select -Expand distinguishedName

    #[2] Parse data: format membersAll like SQLLogins
     ForEach ($member in $membersAll) {
        $memberSplit = $member -split "," -replace '(CN=|DC=)'
        $members += @($memberSplit[4] + "\" + $memberSplit[0]) 
    }

    #[3,4] Process data, Pass data
        $global:membersTrue = Compare-Object @($SQL_Logins) @($members) -Passthru -ExcludeDifferent
        $global:membersFalse = $members.Where{$_ -notin $membersTrue} 
        $global:MembersTrueGlobal += $membersTrue 
        
    }

In [ ]:
##[1.5]## Set variables - Define Functions 

   function Get-Dismissed() {

    #[1] Parse data: Split logins
        $domain, $login = $SQL_Login -split "\\"

    #[2, 3, 4] Pull data, Process data, Pass Data: add SQL_Login to dismissed if adUser does not have Enabled=True or is not found in AD
        try {$ad_User_Enabled_Status = get-aduser $login -Server "$domain.com" | Select -Expand Enabled
            If ($ad_User_Enabled_Status -ne "true") {$global:Dismissed += $SQL_Login}}
        catch [Microsoft.ActiveDirectory.Management.ADIdentityResolutionException] {
            Write "$SQL_Login not found in $domain.com - consider as dismissed"
            $global:Dismissed += $SQL_Login 
        }
    }

In [ ]:
#[2]# Get groups members
    ForEach ($AD_group in $AD_Groups) {
        Get-GroupMembers
        Write `n"Logins that are members of $AD_group`:" ; $membersTrue
        Write `n"Members of $AD_group that are not in SQL Logins:" ; $membersFalse 
    }
    
    $SQL_Logins = $SQL_Logins.Where{$_ -notin $MembersTrue}

In [ ]:
#[3]# Get Dismissed
    Write `n"Looking for dismissed employees..."
    ForEach ($SQL_Login in $SQL_Logins) { Get-Dismissed }
    
    Write `n"Dismissed:"; $Dismissed
    $SQL_Logins = $SQL_Logins.Where{$_-notin $Dismissed}

In [ ]:
#[4]# Write results

    #[1] Pull data
        $DBs = Invoke-Sqlcmd -ServerInstance $SQL_server `
        -Query "SELECT SUSER_SNAME(owner_sid) AS login_name, name AS database_name FROM sys.databases"

        $Jobs = Invoke-Sqlcmd -ServerInstance $SQL_server `
        -Query "SELECT l.name AS login_name, j.name AS job_name FROM msdb.dbo.sysjobs j JOIN sys.syslogins l ON j.owner_sid = l.sid"

In [ ]:
    #[2] Process data
        $DBs = $DBs.Where{$_.login_name -notin "sa"}
        
        $Jobs = $Jobs.Where{$_.login_name `
            -notin "sa" `
            -and $_.login_name -notin "NT AUTHORITY\SYSTEM"}

In [ ]:
    #[3] Print data
        Write `n"SQL logins that are not members of groups and are not dismissed:"; 
        $SQL_Logins

In [ ]:
        Write `n"DBs ownership:"; 
        $DBs | Out-String

In [ ]:
        Write `n"Jobs ownership:" ; 
        $Jobs | Out-String